In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from sparsepixels.utils import *

#### ---> extract raw images and necessary info from official samples
#run_batch(which="inclusive", overwrite_outputs=True, store_all=True)
#run_batch(which="nue", overwrite_outputs=True, store_all=True)

#### ---> slim files with filtering by min signal hits
#incl_in = [f"datasets/neutrino/data_inclusive_plane{p}.h5" for p in (0,1,2)]
#incl_out = [f"datasets/neutrino/data_inclusive_plane{p}_slim_500.h5" for p in (0,1,2)]
#nue_in = [f"datasets/neutrino/data_nue_plane{p}.h5" for p in (0,1,2)]
#nue_out = [f"datasets/neutrino/data_nue_plane{p}_slim_500.h5" for p in (0,1,2)]
#for inp, outp in zip(incl_in, incl_out):
    #write_slim_plane_ge_threshold(inp, outp, threshold=500, overwrite=True, verbose=True)
#for inp, outp in zip(nue_in, nue_out):
    #write_slim_plane_ge_threshold(inp, outp, threshold=500, overwrite=True, verbose=True)

#### ---> extract sig/bkg windows into training-ready files
#write_patches_from_slim_file("datasets/neutrino/data_inclusive_plane2_slim_500.h5", win_t=256, win_w=512, num_bkg=5)
#write_patches_from_slim_file("datasets/neutrino/data_inclusive_plane0_slim_500.h5", win_t=256, win_w=512, num_bkg=5)
#write_patches_from_slim_file("datasets/neutrino/data_inclusive_plane1_slim_500.h5", win_t=256, win_w=512, num_bkg=5)
#write_patches_from_slim_file("datasets/neutrino/data_nue_plane0_slim_500.h5", win_t=256, win_w=512, num_bkg=5)
#write_patches_from_slim_file("datasets/neutrino/data_nue_plane1_slim_500.h5", win_t=256, win_w=512, num_bkg=5)
#write_patches_from_slim_file("datasets/neutrino/data_nue_plane2_slim_500.h5", win_t=256, win_w=512, num_bkg=5)

In [ ]:
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='sig', point_size=3)
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='bkg', point_size=3)
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='sigbkg', point_size=3)

#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='sig', point_size=3)
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='bkg', point_size=3)
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='sigbkg', point_size=3)

#for i in range(10):
    #plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=i, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
    #plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=i, p='sigbkg', point_size=3)
    #plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=i, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
    #plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=i, p='sigbkg', point_size=3)

run_window_demo("datasets/neutrino/data_inclusive_plane2_slim_500.h5", first_N=5, win_t=256, win_w=512, num_bkg=5, cmap='gist_ncar', vmin=0, vmax=100)
#run_window_demo("datasets/neutrino/data_nue_plane2_slim_500.h5", first_N=5, win_t=256, win_w=512, num_bkg=5, cmap='gist_ncar', vmin=0, vmax=100)

In [ ]:
preview_patches("datasets/neutrino/data_inclusive_plane2_slim_500_patches_sig_t256w512.h5", max_examples=5, start_idx=0, cmap='gist_ncar', vmin=0, vmax=100)
preview_patches("datasets/neutrino/data_inclusive_plane2_slim_500_patches_bkg_t256w512.h5", max_examples=5, start_idx=0, cmap='gist_ncar', vmin=0, vmax=100)